In [3]:
pickle_in = open("database_past_asks_bids_COP_BTC.pickle","rb")
database_past_asks_bids=pickle.load(pickle_in)
database_past_asks_bids

,id,account_id,amount,created_at,fee_currency,limit,market_id,original_amount,paid_fee,price_type,state,total_exchanged,traded_amount,type,json,source,uuid
0,28497160,98514,0.00441,2020-05-08 04:24:22.375000,COP,36799994.0,BTC-COP,0.00441,0.0,limit,canceled,0.0,0.0,Ask,"{'id': 28497160, 'market_id': 'BTC-COP', 'acco...",NaN,NaN
1,28497079,98514,0.00441,2020-05-08 04:22:05.663000,COP,36799994.0,BTC-COP,0.00523,105.61,limit,canceled,30176.0,0.00082,Ask,"{'id': 28497079, 'market_id': 'BTC-COP', 'acco...",NaN,NaN
2,28496980,98514,0.00523,2020-05-08 04:18:29.903000,COP,36799994.0,BTC-COP,0.00523,0.0,limit,canceled,0.0,0.0,Ask,"{'id': 28496980, 'market_id': 'BTC-COP', 'acco...",NaN,NaN
3,28496394,98514,0.00037,2020-05-08 04:00:47.630000,BTC,36178048.35,BTC-COP,0.00037,0.0,limit,canceled,0.0,0.0,Bid,"{'id': 28496394, 'market_id': 'BTC-COP', 'acco...",NaN,NaN
4,28496389,98514,0.00037,2020-05-08 04:00:45.495000,COP,36532952.0,BTC-COP,0.00037,0.0,limit,canceled,0.0,0.0,Ask,"{'id': 28496389, 'market_id': 'BTC-COP', 'acco...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111949,156333439,98514,0.00605,2021-10-21T14:37:05.923Z,COP,236418846.98,BTC-COP,0.00605,0.0,limit,canceled,0.0,0.0,Ask,NaN,None,3db1fabc-4c52-4e92-b337-c0652effd3a9
111950,156333994,98514,0.00605,2021-10-21T14:38:45.466Z,COP,236410000.0,BTC-COP,0.00605,0.0,limit,canceled,0.0,0.0,Ask,NaN,None,09f72ef8-de8f-4013-88c7-edfa88cc993b
111951,156334522,98514,0.00605,2021-10-21T14:40:23.353Z,COP,236409835.0,BTC-COP,0.00605,0.0,limit,canceled,0.0,0.0,Ask,NaN,None,f9f57e61-8e71-49f9-b410-440fc22eeb3d
111952,156335071,98514,0.00605,2021-10-21T14:42:03.649Z,COP,236409609.95,BTC-COP,0.00605,0.0,limit,canceled,0.0,0.0,Ask,NaN,None,49cc34ee-36db-491f-b3b5-85b2e62973eb


# LIBRARIES

In [ ]:
# !pip install surbtc

In [2]:
import hmac
import time
import json
import base64
import pickle
import surbtc
import smtplib
import requests
import numpy as np
import pandas as pd
import configparser
import requests.auth
import datetime as dt
from datetime import datetime
from email.mime.text import MIMEText
from trading_api_wrappers import Buda
from email.mime.multipart import MIMEMultipart

# CHECKING MONEY

In [ ]:
def get_config(category, key):
    return config[category][key]

#_____

def get_config_market(category, key):
    return config_market[category][key]

#_____

def enviar_alerta(subject, msg, owners):

    me = get_config('owner_information','ME')
    password = get_config('owner_information','PASSWORD')

    #___Create email object
    
    email = MIMEMultipart('alternative')
    
    #___Set parameters
    
    email['Subject'] = subject
    email['From'] = me
    body = MIMEText('<html><body><p>{}</p></body></html>'.format(msg), 'html')

    #___Attach body
    
    email.attach(body)

    #___Create connection
    
    with smtplib.SMTP_SSL('smtp.gmail.com') as connection:

        connection.login(me, password)
        
        #___Send email
        
        for owner in owners:
            email['To'] = owner
            connection.sendmail(me, owner, email.as_string())

#_____

class BudaHMACAuth(requests.auth.AuthBase):
    """Adjunta la autenticación HMAC de Buda al objeto Request."""

    def __init__(self, api_key: str, secret: str):
        self.api_key = api_key
        self.secret = secret

    def get_nonce(self) -> str:
        # 1. Generar un nonce (timestamp en microsegundos)
        return str(int(time.time() * 1e6))

    def sign(self, r, nonce: str) -> str:
        # 2. Preparar string para firmar
        components = [r.method, r.path_url]
        if r.body:
            encoded_body = base64.b64encode(r.body).decode()
            components.append(encoded_body)
        components.append(nonce)
        msg = ' '.join(components)
        # 3. Obtener la firma
        h = hmac.new(key=self.secret.encode(),
                        msg=msg.encode(),
                        digestmod='sha384')
        signature = h.hexdigest()
        return signature

    def __call__(self, r):
        nonce = self.get_nonce()
        signature = self.sign(r, nonce)
        # 4. Adjuntar API-KEY, nonce y firma al header del request
        r.headers['X-SBTC-APIKEY'] = self.api_key
        r.headers['X-SBTC-NONCE'] = nonce
        r.headers['X-SBTC-SIGNATURE'] = signature
        return r    

#_____
    
def getCRYinAccount():
    global client
    global CRYPT
    balance = client.balance(CRYPT)
    return balance.amount.amount

#_____

def getMONinAccount():
    global client
    global balanceMON
    global MONEY
    balance = client.balance(MONEY)
    return balance.amount.amount

#_____

def create_connection_buda():
    global API_KEY
    global API_SECRET
    return Buda.Auth(API_KEY, API_SECRET)

#_____REQUEST ORDER BOOK

def request_order_book():
    global URL
    global API_KEY
    global API_SECRET
    time.sleep(1)
    with requests.get(URL, auth=BudaHMACAuth(API_KEY, API_SECRET)) as r:
        order_book = r.json()
        if ( order_book != None and 'order_book' in order_book ):
            return order_book['order_book']
        else:
            print('Order book no llega completo')

# SEND MAIL

In [ ]:
hourCounter=0

initial_money_cop_btc=2165000
initial_money_cop_bch=440233
initial_money_cop_eth=279628
initial_money_cop_ltc=318019

initial_money_ars_btc=907887
initial_money_ars_bch=945424
initial_money_ars_eth=668229
initial_money_ars_ltc=1869534

initial_money_pen_btc=876862
initial_money_pen_bch=361261
initial_money_pen_eth=617417
initial_money_pen_ltc=563643

initial_money_clp_btc=3599051
initial_money_clp_bch=850000
initial_money_clp_eth=1339933
initial_money_clp_ltc=241724


initial_exchange_cop_ltc=33779071
initial_exchange_cop_btc=35898000
initial_exchange_cop_eth=33779071
initial_exchange_cop_bch=33779071

initial_exchange_ars_ltc=35000000
initial_exchange_ars_btc=31889995
initial_exchange_ars_eth=35299001
initial_exchange_ars_bch=33415181

initial_exchange_pen_ltc=33898000
initial_exchange_pen_btc=32302795
initial_exchange_pen_eth=35000000
initial_exchange_pen_bch=33950018

initial_exchange_clp_ltc=33779071
initial_exchange_clp_btc=32302795
initial_exchange_clp_eth=33779071
initial_exchange_clp_bch=31875000

MARKET_LIST=["cop_btc","clp_bch","pen_eth","clp_btc","ars_eth","cop_eth","pen_btc","cop_bch","clp_eth","pen_ltc","ars_btc"]

#_____

while True:
    
    try:
    
        if (float(datetime.now().hour) == 0.0):

            hourCounter=1

        if (float(datetime.now().hour) == 1.0) and hourCounter==1:

            contador=0
            hourCounter=0

            total_cop=0
            total_ars=0
            total_clp=0
            total_pen=0

            dataFrame=pd.DataFrame()

            #_____

            market_list_1=["cop_btc","clp_bch","pen_eth","ars_ltc"]
            market_list_2=["cop_ltc","clp_btc","pen_bch","ars_eth"]
            market_list_3=["cop_eth","clp_ltc","pen_btc","ars_bch"]
            market_list_4=["cop_bch","clp_eth","pen_ltc","ars_btc"]

            for market in MARKET_LIST:

                if market in market_list_1:

                    config_market = configparser.ConfigParser()
                    config_market.sections()
                    config_market.read('config'+'_'+"cop_btc"+'.ini')
                    API_KEY = get_config_market('buda', 'API_KEY')
                    API_SECRET = get_config_market('buda', 'API_SECRET')
                    client = create_connection_buda()

                if market in market_list_2:

                    config_market = configparser.ConfigParser()
                    config_market.sections()
                    config_market.read('config'+'_'+"ars_eth"+'.ini')
                    API_KEY = get_config_market('buda', 'API_KEY')
                    API_SECRET = get_config_market('buda', 'API_SECRET')
                    client = create_connection_buda()

                if market in market_list_3:

                    config_market = configparser.ConfigParser()
                    config_market.sections()
                    config_market.read('config'+'_'+"pen_btc"+'.ini')
                    API_KEY = get_config_market('buda', 'API_KEY')
                    API_SECRET = get_config_market('buda', 'API_SECRET')
                    client = create_connection_buda()

                if market in market_list_4:

                    config_market = configparser.ConfigParser()
                    config_market.sections()
                    config_market.read('config'+'_'+"clp_eth"+'.ini')
                    API_KEY = get_config_market('buda', 'API_KEY')
                    API_SECRET = get_config_market('buda', 'API_SECRET')
                    client = create_connection_buda()

                config_market = configparser.ConfigParser()
                config_market.sections()
                config_market.read('config'+'_'+market+'.ini')
                MINE=float(get_config_market('buda', 'MINE'))

                pickle_in = open("database_past_asks_bids_"+market.upper()+".pickle","rb")
                database_past_asks_bids=pickle.load(pickle_in)

                #_____

                database_past_asks_bids.traded_amount=database_past_asks_bids.traded_amount.astype(float)
                database_past_asks_bids.total_exchanged=database_past_asks_bids.total_exchanged.astype(float)
                database_past_asks_bids['created_at']= pd.to_datetime(database_past_asks_bids['created_at'])

                #_____

                database_past_asks_bids["AMOUNT_BALANCE"]=np.where(database_past_asks_bids.type=="Ask",database_past_asks_bids.traded_amount,-database_past_asks_bids.traded_amount)
                database_past_asks_bids["MONEY_BALANCE"]=np.where(database_past_asks_bids.type=="Ask",database_past_asks_bids.total_exchanged,-database_past_asks_bids.total_exchanged)

                #_____

                database_past_asks_bids=database_past_asks_bids.loc[database_past_asks_bids.total_exchanged>0.0]

                if MINE!=1:
                    database_past_asks_bids=database_past_asks_bids.loc[database_past_asks_bids.created_at>="2020-05-20"]

                #_____

                amount_balance=round(sum(database_past_asks_bids.AMOUNT_BALANCE.values),5)

                while abs(amount_balance)>0.0:
                    database_past_asks_bids=database_past_asks_bids.iloc[:len(database_past_asks_bids)-1]
                    amount_balance=round(sum(database_past_asks_bids.AMOUNT_BALANCE.values),5)

                #_____

                money_balance=round(sum(database_past_asks_bids.MONEY_BALANCE.values),5)

                if "cop" in market:
                    total_cop=total_cop+money_balance
                if "ars" in market:
                    total_ars=total_ars+money_balance
                if "clp" in market:
                    total_clp=total_clp+money_balance
                if "pen" in market:
                    total_pen=total_pen+money_balance

                btc_balance=float(pd.DataFrame(client.quotation_market(amount=money_balance,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])

                #_____

                dataFrame.at[contador,"CURRENCY"]=market[0:3].upper()
                dataFrame.at[contador,"CRYPTO"]=market[4:].upper()
                dataFrame.at[contador,"INITIAL MONEY (COP)"]="${:,.2f}".format(float(globals()["initial_money_"+market]))

                #

                if "cop" in market:
                    dataFrame.at[contador,"UTILITY (COP)"]="${:,.2f}".format(money_balance)
                else:
                    mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=money_balance,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                    money_balance=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                    dataFrame.at[contador,"UTILITY (COP)"]="${:,.2f}".format(money_balance)

                #

                dataFrame.at[contador,"UTILITY MARGIN (COP)"]="{:,.2f}%".format((money_balance/float(globals()["initial_money_"+market]))*100.0)

                #_____

                if market in market_list_1:

                    URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

                    my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
                    my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount

                    if "cop" in market:
                        try:
                            crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            crypt_to_cop=0.0
                        total_money=my_market_mon+crypt_to_cop
                    else:
                        try:
                            mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                            btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            btc_to_cop=0.0
                        try:
                            crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            crypt_to_cop=0
                        total_money=crypt_to_cop+btc_to_cop

                    dataFrame.at[contador,"MARKET MONEY (COP)"]="${:,.2f}".format(total_money)

                if market in market_list_2:

                    URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

                    my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
                    my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount

                    if "cop" in market:
                        try:
                            crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            crypt_to_cop=0.0
                        total_money=my_market_mon+crypt_to_cop
                    else:
                        try:
                            mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                            btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            btc_to_cop=0.0
                        try:
                            crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            crypt_to_cop=0
                        total_money=crypt_to_cop+btc_to_cop

                    dataFrame.at[contador,"MARKET MONEY (COP)"]="${:,.2f}".format(total_money)

                if market in market_list_3:

                    URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

                    my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
                    my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount

                    if "cop" in market:
                        try:
                            crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            crypt_to_cop=0.0
                        total_money=my_market_mon+crypt_to_cop
                    else:
                        try:
                            mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                            btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            btc_to_cop=0.0
                        try:
                            crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            crypt_to_cop=0
                        total_money=crypt_to_cop+btc_to_cop

                    dataFrame.at[contador,"MARKET MONEY (COP)"]="${:,.2f}".format(total_money)

                if market in market_list_4:

                    URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

                    my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
                    my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount

                    if "cop" in market:
                        try:
                            crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            crypt_to_cop=0.0
                        total_money=my_market_mon+crypt_to_cop
                    else:
                        try:
                            mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                            btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            btc_to_cop=0.0
                        try:
                            crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
                        except:
                            crypt_to_cop=0
                        total_money=crypt_to_cop+btc_to_cop

                    dataFrame.at[contador,"MARKET MONEY (COP)"]="${:,.2f}".format(total_money)

                #_____

                contador=contador+1

            total_cop_draw=0
            total_btc_draw=0

            for i in range(0,len(dataFrame)):
                x=dataFrame.at[i,"UTILITY (COP)"].replace("$","")
                x=float(x.replace(",",""))
                total_cop_draw=total_cop_draw+x

            dataFrame=dataFrame[["CURRENCY","CRYPTO","MARKET MONEY (COP)","INITIAL MONEY (COP)","UTILITY (COP)","UTILITY MARGIN (COP)"]]
            dataFrame.reset_index(drop=True,inplace=True)
            lista_ordenar=[]
            for i in range(0,len(dataFrame)):
                x=dataFrame.at[i,"UTILITY (COP)"].replace("$","")
                x=float(x.replace(",",""))
                lista_ordenar.append(x)
            dataFrame["ORDER"]=lista_ordenar
            dataFrame=dataFrame.sort_values(by=["ORDER"],ascending=False)
            dataFrame.drop(["ORDER"],axis=1,inplace=True)
            dataFrame.reset_index(drop=True,inplace=True)
            dataFrame_1=dataFrame.copy()

            # #_____

            config = configparser.ConfigParser()
            config.sections()
            config.read('config.ini')

            dataFrame=dataFrame.to_html()
            subject = "SpreadNet: Daily Update"
            msg = "{}<br>Net Profit (COP): <b>${:,.2f}</b>".format(dataFrame,total_cop_draw)
            owners = json.loads(config.get('owner_information','OWNERS'))
            enviar_alerta(subject, msg, owners)
            # enviar_alerta(subject, msg, ["andrescobar6@gmail.com"])
            
    except:
        
        time.sleep(300)

# MANUAL CHECK

columnList=["BTC","ETH","LTC","BCH","COP","ARS","PEN","CLP"]
indexList=["ANDRES","JULIANA","FELIPE","IVAN"]

summaryDataFrame=pd.DataFrame(columns=columnList,index=indexList)

myTotalBtc=0.0

#_____

MARKET_LIST=["cop_btc","clp_bch","pen_eth","clp_btc","ars_eth","cop_eth","pen_btc","cop_bch","clp_eth","pen_ltc","ars_btc"]

#_____

market_list_1=["cop_btc","clp_bch","pen_eth","ars_ltc"] #ANDRÉS
market_list_2=["cop_ltc","clp_btc","pen_bch","ars_eth"] #FELIPE
market_list_3=["cop_eth","clp_ltc","pen_btc","ars_bch"] #JULIANA
market_list_4=["cop_bch","clp_eth","pen_ltc","ars_btc"] #IVAN

for market in MARKET_LIST:

    if market in market_list_1:

        config_market = configparser.ConfigParser()
        config_market.sections()
        config_market.read('config'+'_'+"cop_btc"+'.ini')
        API_KEY = get_config_market('buda', 'API_KEY')
        API_SECRET = get_config_market('buda', 'API_SECRET')
        client = create_connection_buda()

    if market in market_list_2:

        config_market = configparser.ConfigParser()
        config_market.sections()
        config_market.read('config'+'_'+"ars_eth"+'.ini')
        API_KEY = get_config_market('buda', 'API_KEY')
        API_SECRET = get_config_market('buda', 'API_SECRET')
        client = create_connection_buda()

    if market in market_list_3:

        config_market = configparser.ConfigParser()
        config_market.sections()
        config_market.read('config'+'_'+"pen_btc"+'.ini')
        API_KEY = get_config_market('buda', 'API_KEY')
        API_SECRET = get_config_market('buda', 'API_SECRET')
        client = create_connection_buda()

    if market in market_list_4:

        config_market = configparser.ConfigParser()
        config_market.sections()
        config_market.read('config'+'_'+"clp_eth"+'.ini')
        API_KEY = get_config_market('buda', 'API_KEY')
        API_SECRET = get_config_market('buda', 'API_SECRET')
        client = create_connection_buda()
        
    #_____
    
    
    if market in market_list_1:

        URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

        my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
        my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount
        
        name="ANDRES"
        mone=market.rsplit("_")[0].upper()
        cryp=market.rsplit("_")[1].upper()
        
        if cryp=="BTC":
    
            myTotalBtc=myTotalBtc+my_market_cry
        
        if cryp!="BTC":
    
            myTotalBtc=myTotalBtc+float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=cryp+'-BTC'.lower(),quotation_type="bid_given_size").json).quote_exchanged[0])
            myTotalBtc=myTotalBtc+float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='BTC-'+mone.lower(),quotation_type="ask_given_value").json).base_exchanged[0])

    if market in market_list_2:

        URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

        my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
        my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount
        
        name="FELIPE"
        mone=market.rsplit("_")[0].upper()
        cryp=market.rsplit("_")[1].upper()
        
        if cryp=="BTC":
    
            myTotalBtc=myTotalBtc+my_market_cry
        
        if cryp!="BTC":
    
            myTotalBtc=myTotalBtc+float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=cryp+'-BTC'.lower(),quotation_type="bid_given_size").json).quote_exchanged[0])
            myTotalBtc=myTotalBtc+float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='BTC-'+mone.lower(),quotation_type="ask_given_value").json).base_exchanged[0])

    if market in market_list_3:

        URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

        my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
        my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount
        
        name="JULIANA"
        mone=market.rsplit("_")[0].upper()
        cryp=market.rsplit("_")[1].upper()
        
        if cryp=="BTC":
    
            myTotalBtc=myTotalBtc+my_market_cry
        
        if cryp!="BTC":
    
            myTotalBtc=myTotalBtc+float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=cryp+'-BTC'.lower(),quotation_type="bid_given_size").json).quote_exchanged[0])
            myTotalBtc=myTotalBtc+float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='BTC-'+mone.lower(),quotation_type="ask_given_value").json).base_exchanged[0])

    if market in market_list_4:

        URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

        my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
        my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount
        
        name="IVAN"
        mone=market.rsplit("_")[0].upper()
        cryp=market.rsplit("_")[1].upper()
        
        if cryp=="BTC":
    
            myTotalBtc=myTotalBtc+my_market_cry
        
        if cryp!="BTC":
    
            myTotalBtc=myTotalBtc+float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=cryp+'-BTC'.lower(),quotation_type="bid_given_size").json).quote_exchanged[0])
            myTotalBtc=myTotalBtc+float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='BTC-'+mone.lower(),quotation_type="ask_given_value").json).base_exchanged[0])
        
    #_____
    
    summaryDataFrame.at[name,mone]=my_market_mon
    summaryDataFrame.at[name,cryp]=my_market_cry

myTotalBtc

summaryDataFrame

listaFormato=["COP","ARS","PEN","CLP"]

for i in indexList:
    
    for j in listaFormato:
    
        try:
            summaryDataFrame.at[i,j]="${:,.2f}".format(str(summaryDataFrame.at[i,j]))

conversionList=["ETH","LTC","BCH","COP","ARS","PEN","CLP]

if "cop" in market:
    try:
        crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
    except:
        crypt_to_cop=0.0
else:
    try:
        mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
        btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
    except:
        btc_to_cop=0.0
    try:
        crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
    except:
        crypt_to_cop=0

contador=0
hourCounter=0

total_cop=0
total_ars=0
total_clp=0
total_pen=0

dataFrame=pd.DataFrame()

#_____

market_list_1=["cop_btc","clp_bch","pen_eth","ars_ltc"]
market_list_2=["cop_ltc","clp_btc","pen_bch","ars_eth"]
market_list_3=["cop_eth","clp_ltc","pen_btc","ars_bch"]
market_list_4=["cop_bch","clp_eth","pen_ltc","ars_btc"]

for market in MARKET_LIST:

    if market in market_list_1:

        config_market = configparser.ConfigParser()
        config_market.sections()
        config_market.read('config'+'_'+"cop_btc"+'.ini')
        API_KEY = get_config_market('buda', 'API_KEY')
        API_SECRET = get_config_market('buda', 'API_SECRET')
        client = create_connection_buda()

    if market in market_list_2:

        config_market = configparser.ConfigParser()
        config_market.sections()
        config_market.read('config'+'_'+"ars_eth"+'.ini')
        API_KEY = get_config_market('buda', 'API_KEY')
        API_SECRET = get_config_market('buda', 'API_SECRET')
        client = create_connection_buda()

    if market in market_list_3:

        config_market = configparser.ConfigParser()
        config_market.sections()
        config_market.read('config'+'_'+"pen_btc"+'.ini')
        API_KEY = get_config_market('buda', 'API_KEY')
        API_SECRET = get_config_market('buda', 'API_SECRET')
        client = create_connection_buda()

    if market in market_list_4:

        config_market = configparser.ConfigParser()
        config_market.sections()
        config_market.read('config'+'_'+"clp_eth"+'.ini')
        API_KEY = get_config_market('buda', 'API_KEY')
        API_SECRET = get_config_market('buda', 'API_SECRET')
        client = create_connection_buda()

    config_market = configparser.ConfigParser()
    config_market.sections()
    config_market.read('config'+'_'+market+'.ini')
    MINE=float(get_config_market('buda', 'MINE'))

    pickle_in = open("database_past_asks_bids_"+market.upper()+".pickle","rb")
    database_past_asks_bids=pickle.load(pickle_in)

    #_____

    database_past_asks_bids.traded_amount=database_past_asks_bids.traded_amount.astype(float)
    database_past_asks_bids.total_exchanged=database_past_asks_bids.total_exchanged.astype(float)
    database_past_asks_bids['created_at']= pd.to_datetime(database_past_asks_bids['created_at'])

    #_____

    database_past_asks_bids["AMOUNT_BALANCE"]=np.where(database_past_asks_bids.type=="Ask",database_past_asks_bids.traded_amount,-database_past_asks_bids.traded_amount)
    database_past_asks_bids["MONEY_BALANCE"]=np.where(database_past_asks_bids.type=="Ask",database_past_asks_bids.total_exchanged,-database_past_asks_bids.total_exchanged)

    #_____

    database_past_asks_bids=database_past_asks_bids.loc[database_past_asks_bids.total_exchanged>0.0]

    if MINE!=1:
        database_past_asks_bids=database_past_asks_bids.loc[database_past_asks_bids.created_at>="2020-05-20"]

    #_____

    amount_balance=round(sum(database_past_asks_bids.AMOUNT_BALANCE.values),5)

    while abs(amount_balance)>0.0:
        database_past_asks_bids=database_past_asks_bids.iloc[:len(database_past_asks_bids)-1]
        amount_balance=round(sum(database_past_asks_bids.AMOUNT_BALANCE.values),5)

    #_____

    money_balance=round(sum(database_past_asks_bids.MONEY_BALANCE.values),5)

    if "cop" in market:
        total_cop=total_cop+money_balance
    if "ars" in market:
        total_ars=total_ars+money_balance
    if "clp" in market:
        total_clp=total_clp+money_balance
    if "pen" in market:
        total_pen=total_pen+money_balance

    btc_balance=float(pd.DataFrame(client.quotation_market(amount=money_balance,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])

    #_____

    dataFrame.at[contador,"CURRENCY"]=market[0:3].upper()
    dataFrame.at[contador,"CRYPTO"]=market[4:].upper()
    dataFrame.at[contador,"INITIAL MONEY (COP)"]="${:,.2f}".format(float(globals()["initial_money_"+market]))

    #

    if "cop" in market:
        dataFrame.at[contador,"UTILITY (COP)"]="${:,.2f}".format(money_balance)
    else:
        mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=money_balance,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
        money_balance=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
        dataFrame.at[contador,"UTILITY (COP)"]="${:,.2f}".format(money_balance)

    #

    dataFrame.at[contador,"UTILITY MARGIN (COP)"]="{:,.2f}%".format((money_balance/float(globals()["initial_money_"+market]))*100.0)

    #_____

    if market in market_list_1:

        URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

        my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
        my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount

        if "cop" in market:
            try:
                crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                crypt_to_cop=0.0
            total_money=my_market_mon+crypt_to_cop
        else:
            try:
                mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                btc_to_cop=0.0
            try:
                crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                crypt_to_cop=0
            total_money=crypt_to_cop+btc_to_cop

        dataFrame.at[contador,"MARKET MONEY (COP)"]="${:,.2f}".format(total_money)

    if market in market_list_2:

        URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

        my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
        my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount

        if "cop" in market:
            try:
                crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                crypt_to_cop=0.0
            total_money=my_market_mon+crypt_to_cop
        else:
            try:
                mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                btc_to_cop=0.0
            try:
                crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                crypt_to_cop=0
            total_money=crypt_to_cop+btc_to_cop

        dataFrame.at[contador,"MARKET MONEY (COP)"]="${:,.2f}".format(total_money)

    if market in market_list_3:

        URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

        my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
        my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount

        if "cop" in market:
            try:
                crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                crypt_to_cop=0.0
            total_money=my_market_mon+crypt_to_cop
        else:
            try:
                mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                btc_to_cop=0.0
            try:
                crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                crypt_to_cop=0
            total_money=crypt_to_cop+btc_to_cop

        dataFrame.at[contador,"MARKET MONEY (COP)"]="${:,.2f}".format(total_money)

    if market in market_list_4:

        URL="https://www.buda.com/api/v2/markets/"+market.replace("_","-")+"/order_book.json"

        my_market_cry=client.balance(market.rsplit("_")[1].upper()).amount.amount
        my_market_mon=client.balance(market.rsplit("_")[0].upper()).amount.amount

        if "cop" in market:
            try:
                crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                crypt_to_cop=0.0
            total_money=my_market_mon+crypt_to_cop
        else:
            try:
                mon_to_btc=float(pd.DataFrame(client.quotation_market(amount=my_market_mon,market_id='btc-'+market[0:3],quotation_type="bid_given_value").json).base_balance_change[0])
                btc_to_cop=float(pd.DataFrame(client.quotation_market(amount=mon_to_btc,market_id="btc-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                btc_to_cop=0.0
            try:
                crypt_to_cop=float(pd.DataFrame(client.quotation_market(amount=my_market_cry,market_id=market[4:]+"-cop",quotation_type="ask_given_size").json).quote_balance_change[0])
            except:
                crypt_to_cop=0
            total_money=crypt_to_cop+btc_to_cop

        dataFrame.at[contador,"MARKET MONEY (COP)"]="${:,.2f}".format(total_money)

    #_____

    contador=contador+1

total_cop_draw=0
total_btc_draw=0

for i in range(0,len(dataFrame)):
    x=dataFrame.at[i,"UTILITY (COP)"].replace("$","")
    x=float(x.replace(",",""))
    total_cop_draw=total_cop_draw+x

dataFrame=dataFrame[["CURRENCY","CRYPTO","MARKET MONEY (COP)","INITIAL MONEY (COP)","UTILITY (COP)","UTILITY MARGIN (COP)"]]
dataFrame.reset_index(drop=True,inplace=True)
lista_ordenar=[]
for i in range(0,len(dataFrame)):
    x=dataFrame.at[i,"UTILITY (COP)"].replace("$","")
    x=float(x.replace(",",""))
    lista_ordenar.append(x)
dataFrame["ORDER"]=lista_ordenar
dataFrame=dataFrame.sort_values(by=["ORDER"],ascending=False)
dataFrame.drop(["ORDER"],axis=1,inplace=True)
dataFrame.reset_index(drop=True,inplace=True)
dataFrame_1=dataFrame.copy()

# #_____

config = configparser.ConfigParser()
config.sections()
config.read('config.ini')

dataFrame=dataFrame.to_html()
subject = "SpreadNet: Daily Update"
msg = "{}<br>Net Profit (COP): <b>${:,.2f}</b>".format(dataFrame,total_cop_draw)
# owners = json.loads(config.get('owner_information','OWNERS'))
# enviar_alerta(subject, msg, owners)
enviar_alerta(subject, msg, ["andrescobar6@gmail.com"])